In [4]:
import numpy as np
from PIL import Image

In [5]:
class Filter():
    def filter():
        pass

In [6]:
class Brightness(Filter):
    def __init__(self,image, bright):
        self.image = image
        self.shape = image.shape
        self.bright = bright
        self.new_image = self.image.astype(np.float32)


    def filter(self):
        for row in range(self.shape[0]):
            for elem in range(self.shape[1]):
                for col in range(3):
                    self.new_image[row, elem, col] *= self.bright

        self.new_image = np.rint(np.clip(self.new_image, 0, 255)).astype(np.uint8)
        
        return self.new_image

In [7]:
class Contrast(Filter):
    def __init__(self,image, contrast, mid = 127.5):
        self.image = image
        self.shape = image.shape
        self.contrast = contrast
        self.new_image = self.image.astype(np.float32)
        self.mid = mid


    def filter(self):
        for row in range(self.shape[0]):
            for elem in range(self.shape[1]):
                for col in range(3):
                    self.new_image[row, elem, col] = (self.new_image[row, elem, col]- self.mid)* self.contrast + self.mid

        self.new_image = np.rint(np.clip(self.new_image, 0, 255)).astype(np.uint8)
        return self.new_image

In [8]:
class Blur(Filter):
    def __init__(self,image, size):
        self.image = image
        self.shape = image.shape
        self.size = size
        self.new_image = self.image.astype(np.float32)
        self.neighbor_range = self.size // 2

    def filter(self):
        for row in range(self.shape[0]):
            for elem in range(self.shape[1]):
                for col in range(3):
                    total = 0
                    count = 0
                    for row1 in range(max(0, row - self.neighbor_range), min(self.shape[0], row + self.neighbor_range + 1)):
                        for elem1 in range(max(0, elem - self.neighbor_range), min(self.shape[1], elem + self.neighbor_range + 1)):
                            total += self.image[row1, elem1, col]
                            count += 1
                    self.new_image[row, elem, col] = total / count
        self.new_image = np.rint(np.clip(self.new_image, 0, 255)).astype(np.uint8)
        return self.new_image


In [9]:
class RGB(Filter):
    def __init__(self,image):
        self.image = image
        self.shape = image.shape
        self.new_image = self.image.astype(np.float32)
    
    def filter(self):
        for row in range(self.shape[0]):
            for elem in range(self.shape[1]):
                if self.image[row, elem, 0] > self.image[row, elem, 1] and self.image[row, elem, 0] > self.image[row, elem, 2]:
                    self.new_image[row, elem, 0] = 255
                    self.new_image[row, elem, 1] = 0
                    self.new_image[row, elem, 2] = 0
                elif self.image[row, elem, 0] < self.image[row, elem, 1] and self.image[row, elem, 1] > self.image[row, elem, 2]:
                    self.new_image[row, elem, 0] = 0
                    self.new_image[row, elem, 1] = 255
                    self.new_image[row, elem, 2] = 0
                elif self.image[row, elem, 2] > self.image[row, elem, 1] and self.image[row, elem, 0] < self.image[row, elem, 2]:
                    self.new_image[row, elem, 0] = 0
                    self.new_image[row, elem, 1] = 0
                    self.new_image[row, elem, 2] = 255
                else:
                    self.new_image[row, elem, 0] = 255
                    self.new_image[row, elem, 1] = 255
                    self.new_image[row, elem, 2] = 255

        self.new_image = np.rint(np.clip(self.new_image, 0, 255)).astype(np.uint8)
        return self.new_image                


In [10]:
class Gray(Filter):
    def __init__(self,image):
        self.image = image
        self.shape = image.shape
        self.new_image = self.image.astype(np.float32)
        
    def filter(self):
        for row in range(self.shape[0]):
            for elem in range(self.shape[1]):
                gray = (self.image[row, elem, 0] * 0.299 +
                        self.image[row, elem, 1] * 0.587 +
                        self.image[row, elem, 2] * 0.114)
                self.new_image[row, elem, 0] = gray
                self.new_image[row, elem, 1] = gray
                self.new_image[row, elem, 2] = gray
        self.new_image = np.rint(np.clip(self.new_image, 0, 255)).astype(np.uint8)
        return self.new_image      
        

In [11]:
class Transparency(Filter):
    def __init__(self, image, transparency):
        self.image = image
        self.shape = image.shape

        self.transparency = transparency * (-255 / 100) + 255
        self.transparency = np.clip(self.transparency, 0, 255)
        self.transparency = int(self.transparency)

    def image_alpha(self):
        self.image = self.image.astype(np.uint8)
        img = Image.fromarray(self.image).convert("RGBA")
        datum = img.getdata()

        newData = []
        for item in datum:
            if item[0] == 255 and item[1] == 255 and item[2] == 255:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)
        
        img.putdata(newData)
        self.image = np.array(img)
        return self.image

    def filter(self):
        self.image_alpha()
        
        for row in range(self.shape[0]):
            for elem in range(self.shape[1]):
                if not (self.image[row, elem, 0] == 255 and 
                        self.image[row, elem, 1] == 255 and 
                        self.image[row, elem, 2] == 255 and 
                        self.image[row, elem, 3] == 0):
                    self.image[row, elem, 3] = self.transparency
        
        return self.image.astype(np.uint8)
